In [1]:
import tensorflow as tf
import numpy as np
import cv2
import time
print(f"tensorflow version: {tf.__version__}")

tensorflow version: 2.10.1


Preprocess

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = np.pad(train_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
train_images = np.expand_dims(train_images,-1)
train_images = train_images.astype(np.float32) / 255.0
# print(train_images.dtype)

test_images = np.pad(test_images,[[0,0],[2,2],[2,2]],"constant",constant_values=0)
test_images = np.expand_dims(test_images,-1)
test_images = test_images.astype(np.float32) / 255.0
# print(train_images.dtype)

CALL MODEL

In [13]:
def call(interpreter_path,img):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # print(output_details)
    index = input_details[0]['index']
    shape = input_details[0]['shape']
    # print("當前輸入圖片格式: ",img.shape)
    # print("當前輸入圖片所需格式: ",shape)
    interpreter.set_tensor(index, img.reshape(shape).astype("float32"))
    interpreter.invoke()
    if output_details[0]['shape'].shape==(2,) and output_details[0]['shape'][1]==10:
        predict = interpreter.get_tensor(output_details[0]['index'])
        try:
            params = interpreter.get_tensor(output_details[1]['index'])
            return predict,params
        except IndexError:
            return predict,0
    else:
        predict = interpreter.get_tensor(output_details[1]['index'])
        params = interpreter.get_tensor(output_details[0]['index'])
        return predict,params

# img = cv2.imread("./images/gray.png",-1)
# img = np.expand_dims(img,-1)
# img = np.expand_dims(img,0)
# img = img.astype(np.float32)/255.0
# print(img.shape,img.dtype)

def model_call(folder_address,img):
    start_time=np.zeros(7)
    end_time=np.zeros(7)
    for i in range(1,7):
        print(f"第{i}層")
        if i == 1:
            start_time[i] = time.perf_counter()
            predict, params = call(f"{folder_address}/Exit_Model_{i}.tflite", img)
            end_time[i] = time.perf_counter()
            
        else:
            start_time[i] = time.perf_counter()
            predict, params = call(f"{folder_address}/Exit_Model_{i}.tflite", params)
            end_time[i] = time.perf_counter()
            
        max_softmax = np.max(predict,-1)
        predict_num = np.argmax(predict,-1)
        print(f"最大softmax: {max_softmax},  此層運算時間: {(end_time[i]-start_time[i])*1000}ms")
        print(f"預測數字: {predict_num}")
        if(max_softmax>=0.9):
            break      
    print(f"總運算時間: {(end_time.sum()-start_time.sum())*1000}ms")
    return predict_num


(1, 32, 32, 1) float32


'\nprint("第一層")\npredict, params = call(f"./TFLITE_Models/float16_Models/Exit_Model_1.tflite", img)\nif(np.max(predict,-1)>=0.9):\n    print(f"預測數字: {np.argmax(predict,-1)}")\nelse:\n    print(f"最大softmax: {np.max(predict,-1)}")\n    print("第二層")\n    predict, params = call(f"./TFLITE_Models/float16_Models/Exit_Model_2.tflite", params)\n    if(np.max(predict,-1)>=0.9):\n        print(f"預測數字: {np.argmax(predict,-1)}")\n    else:\n        print(f"最大softmax: {np.max(predict,-1)}")\n        print("第三層")\n        predict, params = call(f"./TFLITE_Models/float16_Models/Exit_Model_3.tflite", params)\n        if(np.max(predict,-1)>=0.9):\n            print(f"預測數字: {np.argmax(predict,-1)}")\n        else:\n            print(f"最大softmax: {np.max(predict,-1)}")\n            print("第四層")\n            predict, params = call(f"./TFLITE_Models/float16_Models/Exit_Model_4.tflite", params)\n            if(np.max(predict,-1)>=0.9):\n                print(f"預測數字: {np.argmax(predict,-1)}")\n            el

手寫識別測試

In [14]:
def show_xy(event,x,y,flags,param):
    global dots, draw,img_gray                    # 定義全域變數
    if flags == 1:
        if event == 1:
            dots.append([x,y])            # 如果拖曳滑鼠剛開始，記錄第一點座標
        if event == 4:
            dots = []                     # 如果放開滑鼠，清空串列內容
        if event == 0 or event == 4:
            dots.append([x,y])            # 拖曳滑鼠時，不斷記錄座標
            x1 = dots[len(dots)-2][0]     # 取得倒數第二個點的 x 座標
            y1 = dots[len(dots)-2][1]     # 取得倒數第二個點的 y 座標
            x2 = dots[len(dots)-1][0]     # 取得倒數第一個點的 x 座標
            y2 = dots[len(dots)-1][1]     # 取得倒數第一個點的 y 座標
            cv2.line(draw,(x1,y1),(x2,y2),(255,255,255),20)  # 畫直線
        cv2.imshow('img', draw)#draw


dots = []   # 建立空陣列記錄座標
w = 320
h = 320
draw = np.zeros((h,w,3), dtype='uint8')   # 建立 420x240 的 RGBA 黑色畫布

while True:
    cv2.imshow('img', draw)
    cv2.setMouseCallback('img', show_xy)
    keyboard = cv2.waitKey(5)                    # 每 5 毫秒偵測一次鍵盤事件
    if keyboard == ord('q'):
        break                                    # 按下 q 就跳出

    if keyboard == ord('n'):
        img_gray = cv2.cvtColor(draw, cv2.COLOR_BGR2GRAY)   # 轉為灰度圖
        img = cv2.resize(img_gray,(32,32))                          # 變更圖片尺寸
        cv2.imwrite(".\images\gray.png",img)
        img = img/255
        img = np.expand_dims(img,0)
        img = np.expand_dims(img,-1)
        
        model_call("./TFLITE_Models/float16_Models",img)
        print("\n")
        draw = np.zeros((h,w,3), dtype='uint8')
    if keyboard == ord('r'):
        draw = np.zeros((h,w,3), dtype='uint8')  # 按下 r 就變成原本全黑的畫布
        cv2.imshow('img', draw)

第1層
最大softmax: [0.9743128],  此層運算時間: 0.8232999999790991ms
預測數字: [3]
總運算時間: 0.8232999999790991ms


第1層
最大softmax: [0.99220794],  此層運算時間: 0.9463999999752559ms
預測數字: [8]
總運算時間: 0.9463999999752559ms


第1層
最大softmax: [0.9814026],  此層運算時間: 0.8532000000514017ms
預測數字: [3]
總運算時間: 0.8532000000514017ms


第1層
最大softmax: [0.79027885],  此層運算時間: 1.025500000082502ms
預測數字: [1]
第2層
最大softmax: [0.5689121],  此層運算時間: 0.3533999999945081ms
預測數字: [0]
第3層
最大softmax: [0.88116044],  此層運算時間: 0.3407000000379412ms
預測數字: [1]
第4層
最大softmax: [0.81716776],  此層運算時間: 0.2705000000560176ms
預測數字: [1]
第5層
最大softmax: [0.7229139],  此層運算時間: 0.4177000000709086ms
預測數字: [1]
第6層
最大softmax: [0.56967276],  此層運算時間: 0.3656999999748223ms
預測數字: [1]
總運算時間: 2.773499999420892ms


第1層
最大softmax: [0.9660375],  此層運算時間: 0.7163999999875159ms
預測數字: [2]
總運算時間: 0.7163999999875159ms


第1層
最大softmax: [0.99048257],  此層運算時間: 0.8187999999336171ms
預測數字: [8]
總運算時間: 0.8187999999336171ms


第1層
最大softmax: [0.7365499],  此層運算時間: 0.9037999999463864ms
預測數字: [9]


KeyboardInterrupt: 

計時方式測試

In [ ]:
"""aa = time.time()
for i in range(1,1000000):
    i += i*i*i
aa1= time.time()
elapsed_time = aa1-aa
print(f"Elapsed time: {elapsed_time} seconds")

aa = time.perf_counter()
for i in range(1,1000000):
    i += i*i*i
aa1= time.perf_counter()
elapsed_time = aa1-aa
print(f"Elapsed time: {elapsed_time} seconds")

aa = time.process_time()
for i in range(1,1000000):
    i += i*i*i
aa1= time.process_time()
elapsed_time = aa1-aa
print(f"Elapsed time: {elapsed_time} seconds")"""

各模型準確度評估